In [1]:
from glob import glob
import os

In [2]:
texts = glob('text/*')
len(texts)

3978

In [3]:
audios = glob('audio/**/**.wav', recursive = True)
speakers = {}
for a in audios:
    speakers[os.path.split(a)[1].replace('.wav', '')] = a
    
len(speakers)

3984

In [4]:
import soundfile as sf
import textgrid

In [5]:
from tqdm import tqdm

directory = 'call-centre-1'

os.system(f'mkdir {directory}')
os.system(f'mkdir {directory}/wav')
os.system(f'mkdir {directory}/text')

0

In [6]:
def split(path):
    new_path = os.path.split(path)[1].replace('.TextGrid', '')
    y, sr = sf.read(speakers[new_path])
    index = 0
    path_textgrid = speakers[new_path]
    tg = textgrid.TextGrid.fromFile(path)
    for i in range(len(tg)):
        for k in range(len(tg[i])):
            try:
                splitted = y[int(sr * tg[i][k].minTime): int(sr * tg[i][k].maxTime)]
                sf.write(f'{directory}/wav/{new_path}-{i}-{k}.wav', splitted, sr)
                with open(f'{directory}/text/{new_path}-{i}-{k}.wav.txt', 'w') as fopen:
                    fopen.write(tg[i][k].mark)
            except Exception as e:
                print('nested exception', e)


In [7]:
split(texts[0])

In [8]:
wavs = glob(f'{directory}/wav/*.wav')
wavs[0]

'call-centre-1/wav/app_0354_0004_phnd_cc-hot-0-43.wav'

In [9]:
import IPython.display as ipd
ipd.Audio(wavs[0])

In [10]:
!cat {wavs[0].replace('/wav/', '/text/') + '.txt'}

your stays as well as your bills on dining with our hotel (ppb) and (uh) after the first year it will cost (uh) seventy five dollars per year (ppb) but (uh) free cancellation [lah] so if it (uh) if you're not satisfied do let us know

In [11]:
def loop(texts):
    texts = texts[0]
    for t in tqdm(texts):
        try:
            split(t)
        except Exception as e:
            print(e)
            pass


In [12]:
loop((texts[:1],))

100%|██████████| 1/1 [00:00<00:00,  1.92it/s]


In [13]:
import mp

mp.multiprocessing(texts, loop, cores = 8, returned = False)

  2%|▏         | 10/497 [00:13<09:45,  1.20s/it]

The file could not be parsed as a Praat text file as it is lacking a proper header.


  4%|▎         | 18/497 [00:29<14:54,  1.87s/it]

Bad entry: 


  6%|▌         | 29/497 [00:49<16:06,  2.07s/it]

...

 82%|████████▏ | 410/497 [09:32<02:21,  1.63s/it]

Bad entry:             xmax = 302.49



 89%|████████▊ | 440/497 [10:24<01:39,  1.74s/it]

'app_0023_3046_phnd_cc-hot'


100%|██████████| 497/497 [12:14<00:00,  1.48s/it]


In [14]:
!du -hs call-centre-1

82G	call-centre-1


In [ ]:
# !tar --use-compress-program="pigz -0 -k " -cf part6-call-centre-1.tar.gz call-centre-1